In [ ]:
#Instalamos las librerias
!pip install surprise
!pip install scikit-surprise
!pip install gradio

In [ ]:
from gradio.layouts import Column
import gradio as gr
from pandas.io.parquet import read_parquet
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD

In [ ]:
df1= read_parquet('/content/drive/MyDrive/rating.parquet', columns=['userId', 'score','id'])
df1.rename(columns = {'id': 'movieId'}, inplace = True)
df1.head(2)

,userId,score,movieId
0,1,1.0,as680
1,1,4.5,ns2186


In [ ]:
df_title= read_parquet('/content/drive/MyDrive/plataformas.parquet',columns=['id','title'])
df_title.rename(columns = {'id': 'movieId'}, inplace = True)
df_title.head(2)

,movieId,title
0,as1,the grand seduction
1,as2,take care good night


In [ ]:
# Definos el objeto de Reader con el intervalo en que va el rating 
reader = Reader(rating_scale=(1, 5))

In [ ]:
# Pasamos de un datafrma de pandas a uno de surprise para usar la libreria
#tiene que tener un orden definido que es: user (raw) ids, the item (raw) ids, and the ratings, in this order.
data = Dataset.load_from_df(df1[['userId', 'movieId', 'score']], reader)

Pasos de un ML
- Dividir el data en train y test
- Instanciar
- Entrenar
- Predecir
- Evaluar

In [ ]:
# Dividir el data en train y test con un tamaño de 0.25
trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
#instanciasmos
model = SVD()

In [ ]:
# Entrenmos el modelo con los datos de entrenamiento
model.fit(trainset)

In [ ]:
# Predecimos

predictions = model.test(testset)

In [ ]:
# Hacemos una prediccion 
user_Id = 1
movie_Id = 'as1'

# Verificar si el id de la película existe en df_title
if movie_Id not in df_title.movieId.values:
    print('La película con el ID', movie_Id, 'no existe.')

# Verificar si el userId del usuario existe
if user_Id not in df1.userId.values:
    print('El usuario con id:', user_Id, 'no existe.')

# Obtener las calificaciones del usuario en cuestión
user_ratings = df1[df1.userId == user_Id]

# Obtener la lista de películas vistas por el usuario
user_movies = list(user_ratings['movieId'])


if movie_Id in user_movies:
    print('El usuario ya ha visto esta película.')
else:
    prediction = model.predict(user_Id, movie_Id)
    # Imprimir la calificación predicha y si se recomienda o no la película
    print('La calificación predicha para la película con el ID', movie_Id, 'para el usuario con el ID', user_Id, 'es:', prediction.est)
    if prediction.est >= 3.5:
        print('Recomendada')
    else:
        print('No recomendada')

La calificación predicha para la película con el ID as1 para el usuario con el ID 1 es: 4.5248566082663055
Recomendada


Evaluacion del modelo 

In [ ]:
from surprise import accuracy

accuracy.rmse(predictions)

RMSE: 1.0013


1.0013177485956397

Optimizacion de hiperparámetros

In [ ]:
from surprise.model_selection import cross_validate
import numpy as np

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128,256]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9654  0.9657  0.9662  0.9657  0.0003  
Fit time          134.99  148.25  138.29  140.51  5.64    
Test time         78.69   86.52   84.15   83.12   3.28    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9665  0.9659  0.9671  0.9665  0.0005  
Fit time          138.91  156.45  168.08  154.48  11.99   
Test time         96.84   95.31   79.88   90.68   7.66    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9677  0.9678  0.9682  0.9679  0.0002  
Fit time          153.98  148.17  150.37  150.84  2.39    
Test time         86.99   83.15   83.91   84.68   1.66    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9711  0.9

In [ ]:
from gradio.layouts import Column
import gradio as gr
from pandas.io.parquet import read_parquet
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD

df_title= read_parquet('/content/drive/MyDrive/plataformas.parquet',columns=['id','title'])
df_title.rename(columns = {'id': 'movieId'}, inplace = True)

df1= read_parquet('/content/drive/MyDrive/rating.parquet', columns=['userId', 'score','id'])
df1.rename(columns = {'id': 'movieId'}, inplace = True)



def get_movie_recommendation(user_Id, movie_Id):

    # Verificar si el id de la película existe en df_title
    if movie_Id not in df_title.movieId.values:
      return('La película con el ID', movie_Id, 'no existe.')

    # Verificar si el userId del usuario existe
    if user_Id not in df1.userId.values:
      return('El usuario con id:', user_Id, 'no existe.')

    # Obtener las calificaciones del usuario en cuestión
    user_ratings = df1[df1.userId == user_Id]

    # Obtener la lista de películas vistas por el usuario
    user_movies = list(user_ratings['movieId'])


    if movie_Id in user_movies:
      return ('El usuario ya ha visto esta película.')
    else:
      prediction = model.predict(user_Id, movie_Id)
       
      if prediction.est >= 3.5:
          return'Super Recomendada',prediction.est
      else:
          return 'Ni la veas, no te va a gustar',prediction.est
  
demo = gr.Interface(fn=get_movie_recommendation, inputs=["number","text"], outputs="text")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>